In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install unrar
!unrar x gdrive/My\ Drive/data/train.rar

In [ ]:
# !pip install os
import os
DIR = '/content/dataset/labels/train'
print (len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

14228


In [ ]:
!rm -rf /content/dataset/labels/train

In [ ]:
!unrar x /content/gdrive/MyDrive/data/val.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/gdrive/MyDrive/data/val.rar

Creating    val                                                       OK
Extracting  val/0003_0662359351_01_WRI-R2_M011.png                         0%  OK 
Extracting  val/0003_0664918633_03_WRI-R1_M011.png                         0%  OK 
Extracting  val/0006_0290481856_01_WRI-R1_M011.png                         0%  OK 
Extracting  val/0008_1128992441_01_WRI-L2_M005.png                         0%  OK 
Extracting  val/0010_0604034401_01_WRI-L1_M014.png                         0%  OK 
Extracting  val/0016_0321123728_02_WRI-L1_F010.png                         0%  OK 
Extracting  val/0028_0576606781_01_WRI-R1_M008.png                         0%  OK 
Extracting  val/0028_0576606836_01_WRI-R2_M008.png                         0%  OK 
Extracting  val/0028_0579537367_03_WRI-R2_M008.png                  

In [ ]:
!unrar x /content/gdrive/MyDrive/data/train labels.rar

In [42]:
%cd F:\FYP DATA\YOLO Datasets\yolov5
!dir

F:\FYP DATA\YOLO Datasets\yolov5
 Volume in drive F is Ammar's Drive.
 Volume Serial Number is 50A0-E696

 Directory of F:\FYP DATA\YOLO Datasets\yolov5

11/21/2022  09:54 PM    <DIR>          .
11/21/2022  09:54 PM    <DIR>          ..
11/21/2022  09:54 PM    <DIR>          .git
08/10/2022  12:39 AM                75 .gitattributes
11/21/2022  09:54 PM    <DIR>          .github
08/10/2022  12:39 AM             3,982 .gitignore
08/10/2022  12:39 AM             1,553 .pre-commit-config.yaml
08/10/2022  12:39 AM             4,964 CONTRIBUTING.md
08/10/2022  12:39 AM    <DIR>          data
08/10/2022  12:39 AM            13,574 detect.py
08/10/2022  12:39 AM            30,287 export.py
08/10/2022  12:39 AM             6,411 hubconf.py
08/10/2022  12:39 AM            35,127 LICENSE
11/21/2022  09:57 PM               236 meta.yaml
11/21/2022  09:54 PM    <DIR>          models
08/10/2022  12:39 AM            15,895 README.md
08/10/2022  12:39 AM             1,087 requirements.txt
08/10/2022 

In [4]:
%git clone https://github.com/ultralytics/yolov5  # clone repo

UsageError: Line magic function `%git` not found.


In [24]:
!pip install -qr requirements.txt 


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\Ammar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [ ]:
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 596 kB 5.1 MB/s 
Setup complete. Using torch 1.12.0+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [5]:
yolo_path = r'F:\FYP DATA\Trained YOLOV5 Model\yolov5'

In [20]:
%cat meta.yaml

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# define number of classes based on YAML
import yaml
with open("meta.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
num_classes

'9'

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [48]:
# time its performance
# %%time
!python train.py --img 320 --batch 16 --epochs 50 --data meta.yaml --cfg ./models/yolov5s.yaml --weights '' --name yolov5s_results  --cache

github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5


train: weights='', cfg=./models/yolov5s.yaml, data=meta.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
fatal: cannot change to 'F:\FYP': No such file or directory
YOLOv5  2022-8-10 Python-3.9.12 torch-1.13.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, tran

In [27]:
# TESTING 

!unrar x "/content/gdrive/MyDrive/Shared Datasets/yolov5.rar"

'unrar' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!unzip "/content/gdrive/MyDrive/Shared Datasets/ISU chest dataset.zip"

Streaming output truncated to the last 5000 lines.
  inflating: images_normalized/2198_IM-0808-2001.dcm.png  
  inflating: images_normalized/2199_IM-0809-0001-0002.dcm.png  
  inflating: images_normalized/219_IM-0799-1001.dcm.png  
  inflating: images_normalized/219_IM-0799-2001.dcm.png  
  inflating: images_normalized/21_IM-0729-1001-0001.dcm.png  
  inflating: images_normalized/21_IM-0729-1001-0002.dcm.png  
  inflating: images_normalized/2200_IM-0811-1001.dcm.png  
  inflating: images_normalized/2200_IM-0811-2001.dcm.png  
  inflating: images_normalized/2201_IM-0811-1002.dcm.png  
  inflating: images_normalized/2202_IM-0811-1001.dcm.png  
  inflating: images_normalized/2202_IM-0811-1002.dcm.png  
  inflating: images_normalized/2203_IM-0812-1001.dcm.png  
  inflating: images_normalized/2203_IM-0812-2001.dcm.png  
  inflating: images_normalized/2204_IM-0813-1001.dcm.png  
  inflating: images_normalized/2204_IM-0813-1002.dcm.png  
  inflating: images_normalized/2205_IM-0814-1001.dcm.pn

In [ ]:
import random
import os
from IPython.display import Image, clear_output  # to display images

chest_imgs_path = "/content/images_normalized"
imgs_len = len(os.listdir(chest_imgs_path))
files = os.listdir(chest_imgs_path)

In [ ]:
i = 0

In [ ]:
rand = random.randint(0, imgs_len)
file = os.path.join(chest_imgs_path, files[rand])
Image(filename=file, width=256)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(file)

In [ ]:
%cd yolov5
!python detect.py --source {file} --weights /content/yolov5/runs/train/yolov5s_results/weights/best.pt
i += 1

[Errno 2] No such file or directory: 'yolov5'
/content/yolov5
detect: weights=['/content/yolov5/runs/train/yolov5s_results/weights/best.pt'], source=/content/images_normalized/2864_IM-1272-1001.dcm.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.7.15 torch-1.12.1+cu113 CPU

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7268094 parameters, 0 gradients
image 1/1 /content/images_normalized/2864_IM-1272-1001.dcm.png: 640x640 1 text, Done. (0.475s)
Speed: 1.9ms pre-process, 475.2ms inference, 1.2ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp2


In [ ]:
from IPython.display import Image, clear_output  # to display images

Image(filename="/content/yolov5/runs/detect/exp" + str(i) + "/" + files[rand], width=416)

Output hidden; open in https://colab.research.google.com to view.